# Pipeline to transform the set of nmdc-schema-compliant mongodb collections to an RDF dataset amenable to SPARQL queries.

Ensure that changes to the code will be import-able in this notebook without needing restart the kernel and thus lose state.

In [1]:
%load_ext autoreload
%autoreload 2

Connect to local dockerized dev environment.

In [2]:
from dotenv import load_dotenv

load_dotenv(".env.localhost")
!env | grep MONGO_HOST

MONGO_HOST=localhost:27018


Initialize a db connection.

In [3]:
from nmdc_runtime.api.db.mongo import get_mongo_db

mdb = get_mongo_db()

Get all populated nmdc-schema collections with entity `id`s.

In [4]:
from nmdc_runtime.util import schema_collection_names_with_id_field

populated_collections = sorted([
    name for name in set(schema_collection_names_with_id_field()) & set(mdb.list_collection_names())
    if mdb[name].estimated_document_count() > 0
])
populated_collections

['biosample_set',
 'data_object_set',
 'extraction_set',
 'field_research_site_set',
 'library_preparation_set',
 'mags_activity_set',
 'metabolomics_analysis_activity_set',
 'metagenome_annotation_activity_set',
 'metagenome_assembly_set',
 'metagenome_sequencing_activity_set',
 'metaproteomics_analysis_activity_set',
 'metatranscriptome_activity_set',
 'nom_analysis_activity_set',
 'omics_processing_set',
 'pooling_set',
 'processed_sample_set',
 'read_based_taxonomy_analysis_activity_set',
 'read_qc_analysis_activity_set',
 'study_set']

Get a JSON-LD context for the NMDC Schema, to serialize documents to RDF.

In [5]:
import json
from pprint import pprint

from linkml.generators.jsonldcontextgen import ContextGenerator
from nmdc_schema.nmdc_data import get_nmdc_schema_definition

context = ContextGenerator(get_nmdc_schema_definition())
context = json.loads(context.serialize())["@context"]

for k, v in list(context.items()):
    if isinstance(v, dict): #and v.get("@type") == "@id":
        v.pop("@id", None) # use nmdc uri, not e.g. MIXS uri
pprint(context)

{'@vocab': 'https://w3id.org/nmdc/',
 'CATH': 'https://bioregistry.io/cath:',
 'CHEBI': {'@prefix': True},
 'CHEMBL.COMPOUND': 'https://bioregistry.io/chembl.compound:',
 'CHMO': {'@prefix': True},
 'COG': 'https://bioregistry.io/cog:',
 'Contaminant': 'http://example.org/contaminant/',
 'CreditAssociation': {},
 'DRUGBANK': 'https://bioregistry.io/drugbank:',
 'EC': 'https://bioregistry.io/eccode:',
 'EFO': 'http://www.ebi.ac.uk/efo/',
 'EGGNOG': 'https://bioregistry.io/eggnog:',
 'ENVO': {'@prefix': True},
 'FBcv': {'@prefix': True},
 'FMA': {'@prefix': True},
 'GO': {'@prefix': True},
 'HMDB': 'https://bioregistry.io/hmdb:',
 'ISA': 'http://example.org/isa/',
 'KEGG.COMPOUND': 'https://bioregistry.io/kegg.compound:',
 'KEGG.ORTHOLOGY': 'https://bioregistry.io/kegg.orthology:',
 'KEGG.REACTION': 'https://bioregistry.io/kegg.reaction:',
 'KEGG_PATHWAY': 'https://bioregistry.io/kegg.pathway:',
 'MASSIVE': 'https://bioregistry.io/reference/massive:',
 'MESH': 'https://bioregistry.io/mes

Ensure `nmdc:type` has a `URIRef` range, i.e. `nmdc:type a owl:ObjectProperty`.

In [6]:
context['type'] = {'@type': '@id'}

Initialize an in-memory graph to store triples, prior to serializing to disk.

In [7]:
from rdflib import Graph

g = Graph()

Define a helper function to speed up triplification process.

In [8]:
def split_chunk(seq, n: int):
    """
    Split sequence into chunks of length n. Do not pad last chunk.
    
    >>> list(split_chunk(list(range(10)), 3))
    [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]
    """
    for i in range(0, len(seq), n):
        yield seq[i : i + n]

Use `rdflib` JSON-LD parsing to ingest mongo docs to in-memory graph.

In [9]:
from toolz import assoc, dissoc
from tqdm.notebook import tqdm

chunk_size = 2_000
total = sum((1 + mdb[name].estimated_document_count() // 2_000) for name in populated_collections)

pbar = tqdm(total=total)

for name in populated_collections:
    print(name)
    docs = [dissoc(doc, "_id") for doc in mdb[name].find()]
    chunks = list(split_chunk(docs, chunk_size))
    for chunk in chunks:
        doc_jsonld = {"@context": context, "@graph": chunk}
        g.parse(data=json.dumps(doc_jsonld), format='json-ld')
        pbar.update(1)
print(f"{len(g):,} triples loaded")

  0%|          | 0/112 [00:00<?, ?it/s]

biosample_set
data_object_set
extraction_set
field_research_site_set
library_preparation_set
mags_activity_set
metabolomics_analysis_activity_set
metagenome_annotation_activity_set
metagenome_assembly_set
metagenome_sequencing_activity_set
metaproteomics_analysis_activity_set
metatranscriptome_activity_set
nom_analysis_activity_set
omics_processing_set
pooling_set
processed_sample_set
read_based_taxonomy_analysis_activity_set
read_qc_analysis_activity_set
study_set
6,546,004 triples loaded


Correct crazy URIs that end with newlines, which messes up graph serialization.

In [10]:
from rdflib import Namespace, RDF, Literal, URIRef

NMDC = Namespace("https://w3id.org/nmdc/")

for s, p, o in g:
    s_str = str(s)
    if s_str.endswith("\n"):
        s_str_fixed = str(s_str)[:-2]
        g.remove((s,p,o))
        g.add((URIRef(s_str_fixed), p,o))
    if isinstance(o, URIRef):
        o_str = str(o)
        if o_str.endswith("\n"):
            o_str_fixed = str(o_str)[:-2]
            g.remove((s,p,o))
            g.add((s, p, URIRef(o_str_fixed)))

Given a schema-collection entity (i.e. one with an `id` and its own mongo document), we want to easily find all other schema-collection entities to which it connects, via any slot.

To do this, we first gather all schema classes that are the type of a schema-collection entity, as well as these class' ancestors.

In [11]:
from linkml_runtime.utils.schemaview import SchemaView

from nmdc_runtime.util import nmdc_schema_view, nmdc_database_collection_instance_class_names

schema_view = nmdc_schema_view()
toplevel_classes = set()
for name in nmdc_database_collection_instance_class_names():
    toplevel_classes |= set(schema_view.class_ancestors(name))
toplevel_classes

{'Activity',
 'Biosample',
 'BiosampleProcessing',
 'CollectingBiosamplesFromSite',
 'DataObject',
 'Extraction',
 'FieldResearchSite',
 'FunctionalAnnotation',
 'FunctionalAnnotationAggMember',
 'GenomeFeature',
 'LibraryPreparation',
 'MagsAnalysisActivity',
 'MaterialEntity',
 'MetabolomicsAnalysisActivity',
 'MetagenomeAnnotationActivity',
 'MetagenomeAssembly',
 'MetagenomeSequencingActivity',
 'MetaproteomicsAnalysisActivity',
 'MetatranscriptomeActivity',
 'NamedThing',
 'NomAnalysisActivity',
 'OmicsProcessing',
 'PlannedProcess',
 'Pooling',
 'ProcessedSample',
 'ReadBasedTaxonomyAnalysisActivity',
 'ReadQcAnalysisActivity',
 'Site',
 'Study',
 'WorkflowExecutionActivity'}

Next, we determine which slots have such a "top-level" class as its range.

In [12]:
slots = schema_view.all_slots()

toplevel_entity_connectors = set()
for k, v in context.items():
    if isinstance(v, dict) and "@type" in v and v["@type"] == "@id":
        if slots[k].range in toplevel_classes and slots[k].domain != "Database":
            toplevel_entity_connectors.add(k)
print(toplevel_entity_connectors)

{'part_of', 'has_input', 'was_informed_by', 'has_output', 'was_generated_by', 'collected_from', 'metagenome_annotation_id'}


Now, let's assert a common symmetric relation for all entities connected by these slots so that we can traverse the graph of top-level entities without needing to specify any specific slot names.

In [13]:
from rdflib import PROV

for s, p, o in g:
    if p.removeprefix(str(NMDC)) in toplevel_entity_connectors:
        g.add((s, PROV.wasInfluencedBy, o))
        g.add((o, PROV.wasInfluencedBy, s))

print(f"{len(g):,} triples in total")

7,017,664 triples in total


Serialize and store as gzipped N-Triples file.

In [14]:
import gzip

with gzip.open('data/nmdc-db.nt.gz', 'wb') as f:
    f.write(g.serialize(format='nt').encode())

Wipe any existing persisted data.

In [15]:
!docker compose up fuseki -d
!docker exec fuseki rm -rf /fuseki-base/nmdc-db.tdb

[+] Running 1/0
 ✔ Container fuseki  Running                                               0.0s 


Ensure data is present to load.

In [16]:
!docker cp data/nmdc-db.nt.gz fuseki:/fuseki-base/

Preparing to copy...Copying to container - 0B5.54MB19.3MB36.1MB54.4MB71.2MB88.5MB103MB119MB134MB150MB167MB183MB198MB214MBSuccessfully copied 215MB to fuseki:/fuseki-base/


Take server down in order to bulk-load data.

In [17]:
!docker compose down fuseki

[+] Running 0/0
 ⠋ Container fuseki  Stopping                                              0.1s 
[+] Running 0/1
 ⠙ Container fuseki  Stopping                                              0.2s 
[+] Running 0/1
 ⠹ Container fuseki  Stopping                                              0.3s 
[+] Running 0/1
 ⠸ Container fuseki  Stopping                                              0.4s 
[+] Running 0/1
 ⠼ Container fuseki  Stopping                                              0.5s 
[+] Running 0/1
 ⠴ Container fuseki  Stopping                                              0.6s 
[+] Running 0/1
 ⠦ Container fuseki  Stopping                                              0.7s 
[+] Running 0/1
 ⠧ Container fuseki  Stopping                                              0.8s 
[+] Running 1/1
 ✔ Container fuseki  Removed                                               0.9s 
[+] Running 2/1
 ✔ Container fuseki              Removed                                   0.9s 
 ! Network nmdc-runtime_defaul

Bulk-load data.

In [18]:
!docker compose run fuseki ./apache-jena-4.9.0/bin/tdbloader --loc /fuseki-base/nmdc-db.tdb /fuseki-base/nmdc-db.nt.gz

20:07:47 INFO  loader          :: -- Start triples data phase
20:07:47 INFO  loader          :: ** Load empty triples table
20:07:47 INFO  loader          :: -- Start quads data phase
20:07:47 INFO  loader          :: ** Load empty quads table
20:07:47 INFO  loader          :: Load: /fuseki-base/nmdc-db.nt.gz -- 2024/03/07 20:07:47 UTC
20:07:48 WARN  riot            :: [line: 36966, col: 92] Bad IRI: Not a valid UUID string: uuid:RMNP-CB-T-55f69c6a-3849-4e45-83ba-1cdfbefc3818
20:07:48 WARN  riot            :: [line: 50585, col: 92] Bad IRI: Not a valid UUID string: uuid:ORNL-CB-B-502e74cf-2cca-4d0c-800c-dc0580d8b54d
20:07:48 WARN  riot            :: [line: 68269, col: 92] Bad IRI: Not a valid UUID string: uuid:LENO-CB-B-26b5381f-14b1-401e-a729-4e6d1b421c37
20:07:49 INFO  loader          :: Add: 100,000 triples (Batch: 70,921 / Avg: 70,921)
20:07:49 WARN  riot            :: [line: 112546, col: 92] Bad IRI: Not a valid UUID string: uuid:UNDE-CB-T-f15c6148-9ebf-41d1-812b-7ba7c15d9d96
20:0

Start up server.

In [19]:
!docker compose up fuseki -d

[+] Running 1/0
 ✔ Container fuseki  Created                                               0.0s 
[+] Running 1/1
 ✔ Container fuseki  Created                                               0.0s 
[+] Running 1/1
 ✔ Container fuseki  Started                                               0.0s 


Now go to <http://localhost:3030/#/dataset/nmdc/query> and SPARQL it up.